In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPooling2D,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.datasets import mnist
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet
import cv2
import random
from keras.utils import to_categorical
import numpy as np
import os
from matplotlib import pyplot

#only uses the cpu
#os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [ ]:
#activate gpu for keras training 
physical_devices = tf.config.experimental.list_physical_devices('GPU')
#print("Num GPUs Available: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
#get both train and validation data from the mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
#shows a sample image from the dataset
def showRandomImage():
    index = random.randint(0,100)
    cv2.imshow('window_name', X_train[index]) 
    cv2.waitKey(0)  
    cv2.destroyAllWindows()  

In [ ]:
#the shape of the image is 28,28 but there are no color channels
X_train[0].shape

In [ ]:
#reshape the inputs
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1)).astype('float32')
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1)).astype('float32')

In [ ]:
#now the shape is 28,28,1
X_train[0].shape

In [ ]:
#normalize the inputs
X_train = X_train / 255
X_test = X_test / 255

In [ ]:
#y labels are the numbers corresponding to the number drawn 
y_train[0]

In [ ]:
#transform the labels so they are in one hot encoded form 
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
y_train[0]

In [ ]:
def add_noise(img):
    '''Add random noise to an image'''
    VARIABILITY = 0.05
    deviation = VARIABILITY*random.random()
    noise = np.random.normal(0, deviation, img.shape)
    img += noise
    np.clip(img, 0., 255.)
    return img

datagen = ImageDataGenerator(preprocessing_function=add_noise)

In [ ]:
datagen.fit(X_train)

In [ ]:
for X, y in datagen.flow(X_train, y_train, batch_size=9):
    for i in range(0, 9):
        pyplot.subplot(330 + 1 + i)
        pyplot.imshow(X[i].reshape(28, 28), cmap=pyplot.get_cmap('gray'))
    # show the plot
    pyplot.show()
    break

In [ ]:
model = Sequential()

model.add(Conv2D(30, (5, 5), input_shape=(28, 28, 1), activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(15, (3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(50, activation='relu'))

model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
#train without image generator
#model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200)

In [ ]:
model.fit(datagen.flow(X_train, y_train, batch_size=32),
          validation_data=(X_test, y_test),
          steps_per_epoch=len(X_train) / 32, epochs=10)

In [ ]:
model.save('model.h5')

In [ ]:
#showRandomImage()